# 数据分析与实践实验四-实验报告

*唐兴平 PB20061145*

- [数据分析与实践实验四-实验报告](#数据分析与实践实验四-实验报告)
  - [实验要求](#实验要求)
  - [实验分析](#实验分析)
  - [实验过程](#实验过程)
  - [实验结论](#实验结论)
  - [附录](#附录)
  - [参考资料](#参考资料)

## 实验要求

> 本次实验针对PICA2015学生调查问卷中的部分数据进行关联规则发掘，预测`repeat`列值为$1$的频繁项目集和关联规则：
> - 选择与 `REPEAT` 列最相关的 5 个特征作为特征集，根据关联规则算法，挖掘这 5 个特征和 REPEAT 构成的频繁项集和关联规则；
> - `'ST127Q01TA', 'ST127Q02TA', 'ST127Q03TA’`仅可以视为一个特征；
> - 计算频繁项集的支持度时就只用考虑数据集中`repeat＝1`的部分，在计算关联规则的置信度时用整体数据集。 # need to be ignored maybe
> - 最小支持度的阈值`minsup=0.6`，最小置信度的阈值`minconf=0.15`；
> - 自行实现关联规则算法，例如`Apriori`或者改进算法；
> - 特征的选取可以是数据集中的原始数据，也可以是自己构建的中间层数据；
> - 自行处理多取值的特征，例如每个取值视为一个`item`；


## 实验分析

1. 同第三次实验对数据进行预处理，并根据第三次实验的阶段性结论，构建一个特征集合；
2. 修改这些特征中的取值的名称，改为易读的形式；
3. 自行实现关联规则算法，支持度和置信度需要由公式给出，后项仅包含`REPEAT=1`的数据；
4. 利用不同的评价指标来衡量这个特征集合的与`REPEAT`的关联性；
5. 按照`国家`字段将数据集进行划分，考察不同国家中特征间的关联性和整体之间的差异。
6. 根据常识和生活经验，对这些特征集合进行直观解释，分析是否存在逻辑谬误；

## 实验过程

### 特征集合选择

数据集的预处理过程同实验三，在此不再赘述。我们根据实验三的阶段性结论，选择如下特征集合：

- `ST127Q01TA`:学生是否在ISCED1中有过复读；
- `HISCED`: 父母双方中的最高学历；
- `CHANGE`: 教育经历中的转校次数总和；
- `ESCS`:家庭经济、社会、文化综合指数；
- `IMMIG`:移民身份；

In [1]:
import pandas as pd
import os,sys

os.chdir(sys.path[0])
file_path="../Exp3/lab3-data-pre.csv"
columns=["ST127Q01TA","HISCED","CHANGE","ESCS","IMMIG","REPEAT"]
dataset_raw=pd.read_csv(file_path,usecols=columns)
dataset=dataset_raw.dropna(axis=0,how='any')

print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25782 entries, 111 to 27165
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ST127Q01TA  25782 non-null  float64
 1   HISCED      25782 non-null  float64
 2   IMMIG       25782 non-null  float64
 3   REPEAT      25782 non-null  float64
 4   CHANGE      25782 non-null  float64
 5   ESCS        25782 non-null  float64
dtypes: float64(6)
memory usage: 1.4 MB
None


### 特征名称替换

考虑到数据集的易读性，我们进行如下的特征取值的替换：
- 对于`REPEAT,IMMIG,ST127Q01TA`字段，直接进行数据类型的转化，并赋予一个现实意义的标签；
- 对于`ESCS`字段，我们假设服从正态分布，取$1\times \sigma=\frac{1}{3}$为阈值，划分为高、中、低三等；
- 对于`HISCED,CHANGE`字段，我们假设为负指数分布，因此划分为无、少数、多三类。

In [2]:
dataset_mapped:pd.DataFrame=pd.DataFrame(columns=columns)

def CHANGE_map(x: float) -> str:
    if x == 0:
        return "CHANGE_0"
    elif x <= 2:
        return "CHANGE_FEW"
    else:
        return "CHANGE_LOT"

def HISCED_map(x: float) -> str:
    if x == 0:
        return "HISC_LOW"
    elif x <= 2:
        return "HISC_MID"
    else:
        return "HISC_HIGH"

def ST127Q01TA_map(x: float) -> str:
    map_dict: dict[float, str] = {1: "ISC_1", 2: "ISC_2", 3: "ISC_3"}
    return map_dict[x]

def REPEAT_map(x: float) -> str:
    map_dict: dict[float, str] = {0: "NO_REPEAT", 1: "REPEAT"}
    return map_dict[x]

def ESCS_map(x: float) -> str:
    threshold:float=1.0/3.0
    if x<-threshold:
        return "ESCS_LOW"
    elif x>threshold:
        return "ESCS_HIGH"
    else:
        return "ESCS_MID"
    
def IMMIG_map(x: float) -> str:
    map_dict: dict[float, str] = {1: "LOCAL", 2: "IMMIG", 3: "IMMIG_CHILD"}
    return map_dict[x]

map_func_dict: dict = {
    "REPEAT": REPEAT_map,
    "IMMIG": IMMIG_map,
    "ESCS": ESCS_map,
    "ST127Q01TA": ST127Q01TA_map,
    "CHANGE": CHANGE_map,
    "HISCED": HISCED_map,
}

for col in dataset.columns:
    dataset_mapped[col] = dataset[col].map(lambda x: map_func_dict[col](x))

dataset_mapped.head()


,ST127Q01TA,HISCED,CHANGE,ESCS,IMMIG,REPEAT
111,ISC_1,HISC_HIGH,CHANGE_0,ESCS_MID,LOCAL,NO_REPEAT
112,ISC_1,HISC_HIGH,CHANGE_0,ESCS_HIGH,LOCAL,NO_REPEAT
113,ISC_1,HISC_HIGH,CHANGE_FEW,ESCS_LOW,IMMIG_CHILD,REPEAT
114,ISC_1,HISC_HIGH,CHANGE_0,ESCS_HIGH,LOCAL,NO_REPEAT
115,ISC_1,HISC_HIGH,CHANGE_0,ESCS_HIGH,LOCAL,NO_REPEAT


### 关联算法实现

本实验采用`FP growth`算法进行关联性分析，具体实现参见[`fp_growth.py`](./fp_growth.py)。

函数的输入：
- `dataset`:数据集，形式为`pd.Dataframe`，所有字段视为`str`型;
- `prediction`:预测结果的列名称，形式为`str`；
- `min_sup`：最小支持度,形式为`float`；
- `min_conf`：最小置信系数形式为`float`；
  
函数的输出为所有满足条件的关联规则集合，形式为`list`，每个成员包含：
- `feature_set`:前项特征集合，形式为`list`
- `conf`:置信度，形式为`float`
- `indicator_set`:各类的评价指标，形式为`dict`

具体算法实现过程：
1. 对输入数据集的形式进行变换，将`pd.Dataframe`转化为`list[list[str]]`形式的数据，每个成员包含一组相关联的记录。
2. 首先建立一颗模式频繁树，也就是一颗字典树。在建立之前，我们需要先对每一个项进行统计，并去除出现次数小于`support count`的项，这里取`min_sup * len(dataset)`；利用字典树我们可以压缩表征原事务集，这是对于原始的`Apriori`算法的优化点；
3. 获取当前项的条件模式基，每一个节点只会有一个前缀；再从条件模式基中获取频繁项集，然后找出这棵频繁模式树的所有路径，整合每个项的频繁项集；
4. 接着利用分治策略，遍历所有的子树，依重复上述过程，直到k-频繁集已经不再满足最小置信度，或者已经遍历了所有的组合；
5. 最后筛选出所有`consquent`为`REPEAT=1`的规则，计算置信度并输出。




In [3]:
from fp_growth import association_rules

rule_list=association_rules(dataset_mapped,"REPEAT",0.6,0.15,"./rules.txt")

if rule_list==None:
  exit(0)

print("{} rules found.".format(len(rule_list)))

100%|██████████| 25782/25782 [00:00<00:00, 123001.96it/s]


frequent item 1:11
frequent item 2:47
frequent item 3:101
frequent item 4:116
frequent item 5:68
frequent item 6:16
result saved in ./rules.txt
221 rules found.


简单查看一下置信度最高的若干个规则：
```python
 1     0.511        ['CHANGE_FEW', 'HISC_MID', 'ESCS_LOW']=>['REPEAT']
 2     0.509        ['CHANGE_FEW', 'LOCAL', 'HISC_MID', 'ESCS_LOW']=>['REPEAT']
 3     0.508        ['CHANGE_FEW', 'HISC_MID']=>['REPEAT']
 4     0.506        ['CHANGE_FEW', 'LOCAL', 'HISC_MID']=>['REPEAT']
 5     0.501        ['CHANGE_FEW', 'HISC_MID']=>['ESCS_LOW', 'REPEAT']
 6     0.498        ['CHANGE_FEW', 'LOCAL', 'HISC_MID']=>['ESCS_LOW', 'REPEAT']
 7     0.485        ['CHANGE_FEW', 'ESCS_LOW']=>['REPEAT']
 8     0.477        ['CHANGE_FEW', 'LOCAL', 'ESCS_LOW']=>['REPEAT']
 9     0.462        ['CHANGE_FEW', 'HISC_HIGH', 'ESCS_LOW']=>['REPEAT']
 10    0.453        ['CHANGE_FEW', 'HISC_HIGH', 'LOCAL', 'ESCS_LOW']=>['REPEAT']
```
一个简单的结论是：转校次数少的本国国民家庭的学生，更倾向于复读。这个结论明显是谬误的。

对于绝对频度较高的特征取值，例如`少数转校次数、本国居民`等，对于`REPEAT=1`有诱导性，因此存在较多这样的规则。如果采样简单的线性模型，利用这些特征作为输入推算可能导致算法的`ACC`较高，但`F1-score`不理想。因此我们需要利用其余指标进行分析，重新判定这些规则对于。

### 评价指标分析

我们利用不同的评价指标，对选取的特征集合进行多角度的分析。除了置信度`confidence`之外，我们还选取四个指标，包括`lift,interest,PS,phi_coef`，定义式如下：

$$
lift=\frac{P(X|Y)}{P(Y)}\\
interest=\frac{P(X,Y)}{P(X)P(Y)}\\
PS=P(X,Y)-P(X)P(Y)\\
\phi-coefficient=\frac{PS}{\sqrt{P(X)[1-P(X)]P(Y)[1-P(Y)]}}
$$

这些指标的计算在`FP growth`算法中得到，实现方法是：得到满足置信度的关联规则后，回到初始的`pd.Dataframe`数据集中，计算各自的特征组合出现的频度，然后计算比例，包括$P(X),P(Y),P(X|Y),P(X,Y)$等参量。具体实现参见[`indicator.py`](./indicator.py)

函数的输入：
- `dataset`：原始数据集，形式为`pd.Dataframe`；
- `rule_list`：关联规则表，形式为`list[tuple]`；

函数的输出为关联规则表的拓展形式，包括每个规则的四个拓展的指标，形式为`dict[str,float]`：
```python
dict({"lift":0.1,"interest":0.2,"PS":0.3,"phi_coef":0.4}) # Example
```

我们对不同的指标进行排序，发现和单纯的置信度衡量相比，不同指标下的排序不一样。综合来看，特征`ST127Q01TA=1`和`REPEAT=1`的关联度最大，这四个指标下的成绩都是最高的，这与我们的预设相一致。综上，我们可以认为：**在ISCED1中有复读情况的、家庭综合指数较低的、父母最高受教育程度较低的学生，复读的概率更大**。

In [4]:
from indicator import indicators

rule_evaluated:list[tuple[set,dict[str,float|int]]]=[]

for rule in rule_list[0:30:5]: #type:ignore
  ancest=rule[0]
  indct=indicators(dataset_mapped,ancest,"REPEAT","REPEAT")
  if indct!=None:
    rule_evaluated.append((ancest,indct))

def check(rule_list:list,indicator:str)->None:
  rule_list.sort(key=lambda x:x[1][indicator])
  print(rule_list[0])  

check(rule_evaluated,"lift")
check(rule_evaluated,"interest")
check(rule_evaluated,"PS")
check(rule_evaluated,"phi_coef")

(frozenset({'CHANGE_FEW', 'HISC_MID', 'ESCS_LOW', 'LOCAL', 'ISC_1'}), {'lift': 0.20932783998346832, 'interest': 1.6240159893222927, 'PS': 0.00386001130920667, 'phi_coef': 0.056338487423420486})
(frozenset({'CHANGE_FEW', 'HISC_MID', 'ISC_1'}), {'lift': 0.26105363828054157, 'interest': 1.580311842452839, 'PS': 0.004600494935905461, 'phi_coef': 0.05955662914063368})
(frozenset({'CHANGE_FEW', 'HISC_MID', 'ESCS_LOW', 'LOCAL', 'ISC_1'}), {'lift': 0.20932783998346832, 'interest': 1.6240159893222927, 'PS': 0.00386001130920667, 'phi_coef': 0.056338487423420486})
(frozenset({'CHANGE_FEW', 'HISC_MID', 'ESCS_LOW', 'LOCAL', 'ISC_1'}), {'lift': 0.20932783998346832, 'interest': 1.6240159893222927, 'PS': 0.00386001130920667, 'phi_coef': 0.056338487423420486})


### 国家比较

我们将数据集按照不同国家（即`CNTRTID`字段）进行划分，共三个国家的数据，分别对选定的数据集进行关联度的分析（预操作和特征的选择同上）。

得到的结果如下：

In [5]:
file_path="../Exp3/lab3-data.csv"
columns=["CNTRYID","ST127Q01TA","HISCED","CHANGE","ESCS","IMMIG","REPEAT"]
dataset_raw=pd.read_csv(file_path,usecols=columns)
#dataset_cnt=dataset_raw.dropna(axis=0,how='any')
dataset_cnt=dataset_raw.drop(index=(dataset_raw.loc[(dataset_raw['CNTRYID']==188.0)].index)) 
for col in columns:
  dataset_cnt[col].fillna(dataset_cnt[col].dropna().median(), inplace=True)
print(dataset_cnt.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42175 entries, 0 to 42175
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CNTRYID     42175 non-null  float64
 1   ST127Q01TA  42175 non-null  float64
 2   HISCED      42175 non-null  float64
 3   IMMIG       42175 non-null  float64
 4   REPEAT      42175 non-null  float64
 5   CHANGE      42175 non-null  float64
 6   ESCS        42175 non-null  float64
dtypes: float64(7)
memory usage: 2.6 MB
None


In [6]:
cnt_list=set(dataset_cnt['CNTRYID'].values)
rule_list_list:list[list]=[]

for cnt in cnt_list:
  dataset=dataset_cnt.loc[dataset_cnt['CNTRYID']==cnt,:]
  dataset=dataset.drop(['CNTRYID'],axis=1)
  dataset_mapped=pd.DataFrame()
  for col in dataset.columns:
      dataset_mapped[col] = dataset[col].map(lambda x: map_func_dict[col](x))
  rule_list=association_rules(dataset_mapped,"REPEAT",0.6,0.15,"./rules/"+str(cnt)+".txt")
  if rule_list!=None:
    rule_list_list.append(rule_list)


100%|██████████| 5831/5831 [00:00<00:00, 144476.53it/s]


frequent item 1:11
frequent item 2:43
frequent item 3:82
frequent item 4:83
frequent item 5:43
frequent item 6:9
result saved in ./rules/484.0.txt


100%|██████████| 4609/4609 [00:00<00:00, 164469.22it/s]


frequent item 1:11
frequent item 2:47
frequent item 3:101
frequent item 4:116
frequent item 5:68
frequent item 6:16
result saved in ./rules/591.0.txt


100%|██████████| 25955/25955 [00:00<00:00, 113194.67it/s]


frequent item 1:11
frequent item 2:47
frequent item 3:101
frequent item 4:116
frequent item 5:68
frequent item 6:16
result saved in ./rules/724.0.txt


100%|██████████| 104/104 [00:00<?, ?it/s]

frequent item 1:10
frequent item 2:38
frequent item 3:72
frequent item 4:73
frequent item 5:38
frequent item 6:8
result saved in ./rules/214.0.txt



100%|██████████| 5676/5676 [00:00<00:00, 101311.86it/s]


frequent item 1:10
frequent item 2:38
frequent item 3:72
frequent item 4:73
frequent item 5:38
frequent item 6:8
result saved in ./rules/152.0.txt


对于每个国家的子数据集，考察置信度较高的部分关联规则。由于数据量的减少，绝对置信度均大幅下降\*：
```python
#CNTRYID=152
 1     0.278        ['ESCS_LOW', 'HISC_MID']=>['REPEAT']
 2     0.278        ['ESCS_LOW', 'HISC_MID', 'CHANGE_0']=>['REPEAT']
 3     0.278        ['ESCS_LOW', 'HISC_MID']=>['CHANGE_0', 'REPEAT']

#CNTRYID=484
 1     1.000        ['ISC_2']=>['REPEAT']
 2     1.000        ['CHANGE_0', 'ISC_2']=>['REPEAT']
 3     1.000        ['ISC_2']=>['CHANGE_0', 'REPEAT']

#CNTRYID=591
 1     0.274        ['CHANGE_FEW', 'HISC_MID', 'ESCS_LOW']=>['REPEAT']
 2     0.274        ['CHANGE_FEW', 'ESCS_LOW']=>['REPEAT']
 3     0.273        ['CHANGE_FEW', 'HISC_MID']=>['REPEAT']

#CNTRYID=724
 1     0.566        ['CHANGE_FEW', 'LOCAL', 'HISC_MID', 'ESCS_LOW']=>['REPEAT']
 2     0.562        ['CHANGE_FEW', 'HISC_MID', 'ESCS_LOW']=>['REPEAT']
 3     0.560        ['CHANGE_FEW', 'LOCAL', 'HISC_MID']=>['REPEAT']
```
*\*其中`CNTRYID=217`由于样本量过少，不满足最小支持度，因此没有得到有效的关联规则*

可以看到，与数据集整体的关联规则集合相比，按照国别进行划分后，选取的 5 个特征的对应取值之间的分布有显著的差异。直观的结论是，
- `CNTRYID=152`的子集中的`较差的综合家庭指数`对于复读的影响是极大的；
- `CNTRYID=484`的子集中，由于数据量太少，导致部分关联规则的置信度到达了$1.0$，这显然是样本分布的误差导致的，不足以形成显著的结论；
- `CNTRYID=591` 的子集和整体的关联规则分布近似，但是没有关于`移民身份`的高置信度关联规则，这可能与国家的移民组成比例有关；
- `CNTRYID=724`的子集和整体的关联规则分布近似，和复读特征关联性最强的为`CHANGE_FEW`，这是由于样本数量占比最大导致的；

综上，按照不同国别进行划分后，形式上关联规则和整体数据集差异较大。如果考虑到样本数量的问题，其中一些与整体数据集矛盾的规则需要被忽略，例如`CNTRYID=484`的子集中，`ISC_2`成为决定是否复读的一个关键的特征取值。

## 实验结论

本实验选取了`"ST127Q01TA","HISCED","CHANGE","ESCS","IMMIG","REPEAT"`共计 5 个特征集合，并映射到了共计 15 个特征的取值，在`min_support=0.6,min_confidence=0.15`的阈值下，得到了满足条件的 220 条关联规则。
- 由于样本数量分布不均问题，频度较高的特征取值对于关联规则的影响较大，规则集合中的出现频次较高，置信度也较高。
- 一个直接的结论是：`转校次数较少的、本国居民的、综合家庭水平中等`的学生更有可能复读，这是涵需商榷的；
- 利用置信度之外的一些指标，例如`lift`指标，我们可以发现与`REPEAT=1`高度相关的特征取值为`ISC_1`，其涵义为：在ISC测试中有过复读的经历，这与现实意义相符。即我们认为：`转校次数较少的、本国居民的、综合家庭水平中等、在ISC测试中有过复读的经历`的学生是极有可能复读的。
- 通过国别划分为 5 个子集，然后分别求出关联规则的集合，与整体相比：我们发现形式上，其中 4 个子集的关联规则和整体的差异较大，其中 1 个与整体很接近。我们认为原因是由于子集的样本数量差异过大导致的，`CNTRYID=724`的子集样本数量最多，对整体的影响也就越大。

## 附录

[FP Growth Algorithm Implementation](./fp_growth.py)

[Indicators for Association Rules](./indicator.py)

[Association Rule List for wholesome Dataset](./rules.txt)

[Association Rule List for specified subset](./rules/152.0.txt)


## 参考资料

1. Apriori 算法简介及 python3实现 [*1]
2. 数据挖掘随笔（二）FP-growth算法——一种用于频繁模式挖掘的模式增长方式(Python实现)[*2]

[*1]: (https://zhuanlan.zhihu.com/p/39918644)

[*2]: (https://zhuanlan.zhihu.com/p/411594391)